In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow

In [ ]:
tensorflow.test.is_gpu_available()

In [ ]:
# 이미지 처리
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# 파일 존재 여부 확인용
import os

In [ ]:
# 훈련용 데이터와 검증용 데이터 분리
from sklearn.model_selection import train_test_split

In [ ]:
# 랜덤숫자 생성
import random

In [ ]:
# 모델
from tensorflow import keras

# 디렉토리 설정

In [ ]:
os.chdir('/content/drive/MyDrive/[perst]데이터셋/MachineLearningProject/01_Cloth Detection Model')

In [ ]:
os.getcwd()

# 변수 목록

In [ ]:
# train_dir_name = "./Dataset/Train-Test_Multiple/Train/"
# test_dir_name = "./Dataset/Train-Test_Multiple/Test/"

train_dir_name = "./Dataset/Train-Test_All/Train/"
test_dir_name = "./Dataset/Train-Test_All/Test/"

img_width = 200
img_height = 200

model_dir_name = './Model/' # 학습 모델이 저장될 디렉토리명

In [ ]:
palette = [0,0,0,
          255,0,0,
          0,255,0,
          0,0,255,
          255,255,0]

In [ ]:
label_string = "0:Background / 1:Outer / 2:Top / 3:Bottom / 4:Onepiece"

# Train 데이터와 Test 데이터 불러오기

In [ ]:
x_train = []
y_train = []

for i in range(0, 8001):
    img_path = train_dir_name + "Image/" + str(i) + ".jpg"
    label_path = train_dir_name + "Annotation/" + str(i) + ".png"

    if not os.path.isfile(img_path):
        continue
    if not os.path.isfile(label_path):
        continue

    # 이미지 파일 로드
    img_file = cv2.imread(img_path)
    if img_file is None:
        print(str(i) + ".jpg File Load Failed!!")
        break

    # 라벨링 파일 로드
    label_file = np.asarray(Image.open(label_path))
    if label_file is None:
        print(str(i) + ".png File Load Failed!!")

    x_train.append(img_file)
    y_train.append(label_file)


    if len(y_train)%500 == 0: # 진행도 확인용
        print(str(len(y_train)) + "개 파일 읽기 완료")


print("Complete")


In [ ]:
x_test = []
y_test = []

for i in range(0, 2001):
    img_path = test_dir_name + "Image/" + str(i) + ".jpg"
    label_path = test_dir_name + "Annotation/" + str(i) + ".png"

    if not os.path.isfile(img_path):
        continue
    if not os.path.isfile(label_path):
        continue

    # 이미지 파일 로드
    img_file = cv2.imread(img_path)
    if img_file is None:
        print(str(i) + ".jpg File Load Failed!!")
        break

    # 라벨링 파일 로드
    label_file = np.asarray(Image.open(label_path))
    if label_file is None:
        print(str(i) + ".png File Load Failed!!")

    x_test.append(img_file)
    y_test.append(label_file)


    if len(y_test)%500 == 0: # 진행도 확인용
        print(str(len(y_test)) + "개 파일 읽기 완료")


print("Complete")


# Train 데이터에서 Validation 데이터 분리

In [ ]:
x_train_ori = x_train[:]
y_train_ori = y_train[:]

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []

x_train, x_val, y_train, y_val = train_test_split(x_train_ori, y_train_ori, test_size=0.25)

# 형태 확인

In [ ]:
x_train = np.array(x_train)
x_train.shape

In [ ]:
y_train = np.array(y_train)
y_train.shape

In [ ]:
x_val = np.array(x_val)
x_val.shape

In [ ]:
y_val = np.array(y_val)
y_val.shape

In [ ]:
x_test = np.array(x_test)
x_test.shape

In [ ]:
y_test = np.array(y_test)
y_test.shape

In [ ]:
plt.imshow(x_train[0])
plt.show()

In [ ]:
print(label_string)

plt.imshow(y_train[0])
plt.clim(0, 4)
plt.colorbar()
plt.show()

# 모델 생성

In [ ]:
input_img = keras.layers.Input(shape=(img_height, img_width, 3))

conv1 = keras.layers.Conv2D(filters=64, kernel_size=5, padding='same', activation='relu')(input_img)
conv1 = keras.layers.Dropout(0.3)(conv1)
conv1 = keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(conv1)
pool1 = keras.layers.MaxPooling2D(2)(conv1)

conv2 = keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(pool1)
conv2 = keras.layers.Dropout(0.3)(conv2)
conv2 = keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(conv2)
pool2 = keras.layers.MaxPooling2D(2)(conv2)

conv3 = keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(pool2)
conv3 = keras.layers.Dropout(0.4)(conv3)
conv3 = keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(conv3)
pool3 = keras.layers.MaxPooling2D(2)(conv3)

conv4 = keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(pool3)
conv4 = keras.layers.Dropout(0.4)(conv4)

up1 = keras.layers.concatenate([keras.layers.UpSampling2D(2)(conv4), conv3], axis=-1)
conv5 = keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(up1)
conv5 = keras.layers.Dropout(0.3)(conv5)
conv5 = keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(conv5)

up2 = keras.layers.concatenate([keras.layers.UpSampling2D(2)(conv5), conv2], axis=-1)
conv6 = keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(up2)
conv6 = keras.layers.Dropout(0.3)(conv6)
conv6 = keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(conv6)

up3 = keras.layers.concatenate([keras.layers.UpSampling2D(2)(conv6), conv1], axis=-1)
conv7 = keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(up3)
conv7 = keras.layers.Dropout(0.3)(conv7)
conv7 = keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(conv7)

output_img = keras.layers.Conv2D(filters=5, kernel_size=1, padding='same', activation='softmax', name='OutputLayer')(conv7)


In [ ]:
poly_seg_model = keras.Model(input_img, output_img)
adam = keras.optimizers.Adam(learning_rate=0.0003)
poly_seg_model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
poly_seg_model.summary()

## 모델 이름은 안바꿔주면 이전 모델에 덮어쓰기되니까 잊지말고 꼭 바꿔주자

In [ ]:
model_path = model_dir_name + 'poly_seg_model_0407_01.keras'

# 모델 학습

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = poly_seg_model.fit(x_train, y_train,
                             epochs=50, verbose=1,
                             validation_data=(x_val, y_val),
                             callbacks=[checkpoint_cb, early_stopping_cb])

print()
print("Complete")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

# 모델 테스트

In [ ]:
saved_model_path = model_dir_name + 'poly_seg_model_0406_06.keras'

saved_model = keras.models.load_model(saved_model_path)

In [ ]:
saved_model.summary()

## 1개 Test 데이터 예측결과 확인

In [ ]:
num = random.randint(0, 1999)
print("Random Number : ", num)

plt.imshow(x_test[num])
plt.show()

In [ ]:
preds = saved_model.predict(x_test[num:num+1])
type(preds)

In [ ]:
preds.shape

In [ ]:
pred_mask = preds.argmax(axis=-1)
pred_mask.shape

In [ ]:
np.unique(pred_mask[0])

In [ ]:
x_test_rgb = cv2.cvtColor(x_test[num], cv2.COLOR_BGR2RGB) # RGB를 RGB 색공간으로 변경

In [ ]:
print(label_string)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(x_test_rgb)
plt.title("Sample Test Image")

plt.subplot(1, 2, 2)
plt.imshow(pred_mask[0])
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("Predict Labeling")

plt.show()

In [ ]:
print(label_string)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(y_test[num])
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("True Labeling")

plt.subplot(1, 2, 2)
plt.imshow(pred_mask[0])
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("Predict Labeling")

plt.show()

In [ ]:
acc = keras.metrics.SparseCategoricalAccuracy()
acc.reset_state()
acc.update_state(pred_mask[0], y_test[num:num+1])
acc.result().numpy()

## 전체 Test 데이터의 정확도 확인

In [ ]:
 saved_model.evaluate(x_test, y_test)

## 남성 모델 이미지의 predict 결과 확인

## 이미지 불러오기

In [ ]:
man_img_path = './Dataset/man_sample.jpg'

man_img = cv2.imread(man_img_path)
if man_img is None:
    print('Image File Load Failed!!')
else:
    plt.imshow(man_img)
    plt.show()

In [ ]:
man_img_array = np.array([man_img])
man_img_array.shape

## 모델 예측

In [ ]:
preds = saved_model.predict(man_img_array[:])
type(preds)

In [ ]:
preds.shape

In [ ]:
pred_mask = preds.argmax(axis=-1)
pred_mask.shape

In [ ]:
np.unique(pred_mask[0])

In [ ]:
plt.imshow(pred_mask[0])
plt.clim(0, 4)
plt.colorbar()
plt.show()

In [ ]:
man_img_rgb = cv2.cvtColor(man_img, cv2.COLOR_BGR2RGB) # RGB를 RGB 색공간으로 변경

In [ ]:
print(label_string)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(man_img_rgb)
plt.title("Man Test Image")

plt.subplot(1, 2, 2)
plt.imshow(pred_mask[0])
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("Predict Labeling")

plt.show()